# 4camping_workspace_ keboola

Tady sepišeme krok za krokem, jaké transformace jsme udělali v keboolu abysme ziskali naše končné data k vizualizace
- prvni pokusi před Hackatonem: 
    -  vybirat a stahovat tabulky z firemniho database
    - vytvorit datový model
    - nahrat data do keboolo
- Hackaton:
    - spojime zbytecne zmatujici a komplikovane tabulky, hodne veci jeste nevime jak fnguje v keboolo co taky neurychlí prace
    - nejakou tabulku jsme potom dali do Tablea, tady taky jsme meli problem, jak updatovat, pokud nejaky data nasledne menime ve keboolo
    - zkusili jsme pridat kraj, okres, psc prodejny, pomoci regex udelat slopuce kde jsou barvy

Vysledek Hackatonu, že začnem znova spolu vytvořit naše zakladnou tabulku, v tomhle workbooku budem sepsat kroky


Extra info:
- pojmenovano tabulky ve storage (CAPS_LOCK)
- tabulky z DB nahrany pres STORAGE
- tabulky ohledne county, region nahrany pres COMPONENT/GOOGLE DRIVE 

- vice o vztazich v datovém modelu
https://lucid.app/lucidchart/5b471e84-a440-4587-a1cf-19c93dbd04c6/edit?invitationId=inv_0c0c0ddc-16c1-424f-ae3e-6f41d12bd1f1&page=wbRCCwA8fE17#

#### 1_4CAMPING_ORDERS_PRODUCT 

VNORENY SELECT UNION PRODUKTU, JOINOVANI S CAT, PROD, ORDERS, SHIPPINGMETHODS

spojit ordes_product nasledne spojime jeste do toho tabulky orders, products, shipping methods a categoris
- Vystupní tabulka bude : ORDER_PRODUCTS_0

In [ ]:
CREATE OR REPLACE TABLE ORDER_PRODUCTS_0 AS ( 
--# select na spojenné tabulky, orders_product, orders, product, chategories, shippinh methods
SELECT 
    OP0.*
    ,O."city" AS CITY
    ,O."date" AS DATES
    ,O."shipping_methods_id" 
    ,O."number" AS ORDER_NUMBER --CISLO OBJEDNÁVKY
    ,P."name_cs" AS PRODUCT_NAME
    ,C."name_cs" AS SUBCATEGORIES_NAME
    ,SM."name_cs" AS SHIPPING_METHOD
    ,SPLIT(C."name_full_cs", '||')[0] AS CATEGORIES --##vytvorime categories pomoci splitem z sloupcu z tabulky category
FROM 
--## 3 orders_product tabulky spojime dohromady
(SELECT
   OP1."id" AS ID
   ,OP1."date_inserted"
   ,OP1."orders_id" AS ORDERS_ID
   ,OP1."products_id" AS PRODUCTS_ID
   ,OP1."variant_name_cs" AS VARIANT_NAME_CS
   ,OP1."quantity" AS QUANTITY
   ,OP1."price_czk" AS PRICE_CZK
   ,OP1."weight" AS WEIGHT
FROM ORDER_PRODUCT_1 AS OP1

UNION ALL

SELECT
    OP2."id"
    ,OP2."date_inserted"
   ,OP2."orders_id"
   ,OP2."products_id"
   ,OP2."variant_name_cs"
   ,OP2."quantity"
   ,OP2."price_czk"
   ,OP2."weight"
FROM ORDER_PRODUCT_2 as OP2

UNION ALL

SELECT
    OP3."id"
    ,OP3."date_inserted"
   ,OP3."orders_id"
   ,OP3."products_id"
   ,OP3."variant_name_cs"
   ,OP3."quantity"
   ,OP3."price_czk"
   ,OP3."weight"
FROM ORDER_PRODUCT_3 as OP3)  AS OP0

INNER JOIN ORDERS AS O --INNER JOIN JE PROTO, ZE TABULKY ORDERS A PRODUCT_ORDERS BYLY STAZENY V RUZNOU DOBU A NE VSECHNY PO JSOU V O
ON OP0.ORDERS_ID = O."id"
LEFT JOIN PRODUCTS AS P
ON OP0.PRODUCTS_ID = P."id"
LEFT JOIN CATEGORIES AS C
ON C."id" = P."categories_id"
LEFT JOIN SHIPPING_METHODS AS SM
ON SM."id" = O."shipping_methods_id"
)

#### 2_4CAMPING_ORDERS_PRODUCT_ADDING_COLUMNS
PRIDAVANI POTREBNYCH SLOUPCU DO TABULKY ORDER_PRODUCTS0 (DATE, ZIP, COUNTY, REGION, COLOR)
- 3 bloky

1. Blok:
- OBOHACENI TAB ORDER_PRODUCT_0 - DATUM

In [ ]:
--upravim tabulku ORDER_PRODUCTS_0 - pridam  nove sloupce: pro praci s datumem
ALTER TABLE ORDER_PRODUCTS_0
ADD COLUMN DAY_NAME VARCHAR(20), DAY_NAME_CS VARCHAR(20), IDAY INTEGER, IYEAR INTEGER, IMONTH INTEGER, IDATE DATE;

--do upravene tabulky orders naplnim prislusne sloupecky
UPDATE ORDER_PRODUCTS_0
SET IDATE = DATE(TO_TIMESTAMP(DATES, 'YYYY-MM-DD HH24:MI:SS'));

UPDATE ORDER_PRODUCTS_0
SET IYEAR = DATE_PART('YEAR', IDATE),
    IMONTH = DATE_PART('MONTH', IDATE),
    IDAY = DATE_PART('DAY', IDATE);

UPDATE ORDER_PRODUCTS_0
SET DAY_NAME = dayname(IDATE),
    DAY_NAME_CS =
    CASE
        WHEN DAY_NAME = 'Sun' THEN 'Neděle'
        WHEN DAY_NAME = 'Mon' THEN 'Pondělí'
        WHEN DAY_NAME = 'Tue' THEN 'Úterý'
        WHEN DAY_NAME = 'Wed' THEN 'Středa'
        WHEN DAY_NAME = 'Thu' THEN 'Čtvrtek'
        WHEN DAY_NAME = 'Fri' THEN 'Pátek'
        WHEN DAY_NAME = 'Sat' THEN 'Sobota'
            END;

2. Blok:
- OBOHACENI TAB ORDER_PRODUCT_0 - BARVY, VELIKOST (!!!! DODELAT), VALUE, UPRAVA DAT.TYPU U PRICE A QNTY

In [ ]:
-- PRIDANI SLOUPCE PRO BARVU - ZISKANI BARVY REGEXEM Z VARIANT_NAME_CS - STEJNYM ZPUSBEM TREBA ZISKAT I VELIKOST - POZOR BARVY TREBA DOLADIT
ALTER TABLE ORDER_PRODUCTS_0 
ADD COLUMN PRODUCT_COLOR VARCHAR(50);

UPDATE ORDER_PRODUCTS_0
SET PRODUCT_COLOR = TRIM(REPLACE(regexp_substr(VARIANT_NAME_CS, 'Barva:([^\/]*)'),'Barva: ', ''));

--DO BUDOUCNA PRIDAME SLOUPEC NA VELIKOST + BARVY JE POTREBA DOLADIT (VIZ. 
--SELECT PRODUCT_COLOR, COUNT(PRODUCT_COLOR) AS VYSKYT
--FROM ORDER_PRODUCTS_0
--WHERE IYEAR = '2022'
--GROUP BY PRODUCT_COLOR
--ORDER BY VYSKYT DESC)

------------------------------------
--MAME PROBLEM PRAZDNYCH HODNOT VE SLOUPCICH, SE KTERYMI CHCEME POCITAT - ZJISTILY JSME, ZE SE JEDNA O CHYBEJICI HODNOTU U PRODUKTU SLUNECNI BRYLE. DOPOCITALY JSME PRUMERNOU CENU (223), ABYCHOM JI DO PRAZDNYCH HODNOT NAHRADILY.
--SELECT PRODUCTS_ID, AVG(PRICE_CZK) AS PRUMER
--FROM ORDER_PRODUCTS_0
--WHERE PRODUCTS_ID = '18409' AND PRICE_CZK <> ''
--GROUP BY PRODUCTS_ID

UPDATE ORDER_PRODUCTS_0
SET PRICE_CZK = '223'
WHERE PRICE_CZK = '';

ALTER TABLE ORDER_PRODUCTS_0 
ADD COLUMN VALUE_ITEM_ORDER INTEGER;

UPDATE ORDER_PRODUCTS_0
SET VALUE_ITEM_ORDER = CAST(PRICE_CZK AS INTEGER) * CAST(QUANTITY AS INTEGER);

3. Blok:
- OBOHACENI TAB ORDER_PRODUCTS_0 O SLOUPEC CARRIER_NAME

In [ ]:
ALTER TABLE ORDER_PRODUCTS_0
ADD COLUMN CARRIER_NAME VARCHAR (20);

UPDATE ORDER_PRODUCTS_0
SET CARRIER_NAME = 
  CASE 
    WHEN shipping_method LIKE '%GLS%' THEN 'GLS' 
    WHEN shipping_method  LIKE '%DPD%' THEN 'DPD' 
    WHEN shipping_method  LIKE '%GEIS%' THEN 'GEIS' 
    WHEN shipping_method  LIKE '%PPL%' THEN 'PPL' 
    WHEN shipping_method  LIKE '%Balíkovna%' THEN 'BALIKOVNA' 
    WHEN shipping_method  LIKE '%iftago%' THEN 'LIFTAGO' 
    WHEN shipping_method  LIKE '%Česká%' THEN 'ČESKÁ POŠTA' 
    WHEN shipping_method  LIKE '%loženka%' THEN 'ULOŽENKA' 
    WHEN shipping_method  LIKE '%Dodo%' THEN 'DODO' 
    WHEN shipping_method  LIKE '%Packeta%' THEN 'PACKETA' 
    WHEN shipping_method  LIKE '%silkovna%' THEN 'PACKETA' 
    WHEN shipping_method  LIKE '%DHL%' THEN 'DHL'
    WHEN shipping_method  LIKE '%SPS%' THEN 'SPS - SK'
    WHEN shipping_method  LIKE '%venská po%' THEN 'SK pošta'
    WHEN shipping_method  LIKE '%test%' THEN 'test'
    ELSE 'OSOBNÍ ODBĚR' 
  END;

#### 3_4CAMPING_ORDERS_PRODUCT_ADDING_ORIGIN
PRIDANI SLOUPCU PUVODU OBJEDNAVKY - OBEC, OKRES, KRAJ - I U OSOBNICH ODBERU UDAJE PRODEJNY